# Building a Database for Crime Reports

The aim of this project is to practice my postgres skills in setting up a database from scratch and use it for storing crime report data. 

The data is the Boston Crime Data. 

Within the user management of the database, I will create two user groups:

* readonly: Users will only be able to read the data. 
* readwrite: Users will be able to read and alter the data but they will not have the permission to delete tables. 

## Creating the Database and Schema

Database named `crime_db` and schema named `crimes` created for storing the tables for containing the crime data. 

In [1]:
import psycopg2

In [2]:
conn = psycopg2.connect("dbname=dq user=dq")
conn.autocommit = True
cur = conn.cursor()

# created the new database 
cur.execute("CREATE DATABASE crime_db;")
conn.close()

In [3]:
# Now the new database has been created, it can be connected too
conn = psycopg2.connect("dbname=crime_db user=dq")
conn.autocommit = True
cur = conn.cursor()

# Created the schema crimes for the data to be stored in
cur.execute("CREATE SCHEMA crimes;")

## Obtaining the Column Names and a Sample of the Data

Being able to access the data's column headers and a sample of the data will help when copying over the data. 

In [17]:
import csv
with open('boston.csv') as file:
    reader = csv.reader(file)
    col_headers = next(reader)
    first_row = next(reader)

## Creating a Function for Analyzing Column Values

Created a function `get_col_set` with the arguements as a CSV file and the column index. The function returns the distinct values for that column. 

Then used the function on each column to find out which columns have a lot of different values. 

The aim was to find columns with limited set of possible values that could be enumerated datatypes. 

In [6]:
def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values

for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


## Analyzing the Maximum Length of the Description Column

Used the above function to compute the maximum description length to decide an appropiate length for that field. 

In [7]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [8]:
descriptions = get_col_set("boston.csv", 2) # description is at index number 2
max_len = 0
for description in descriptions:
    max_len = max(max_len, len(description))
print(max_len)

58


## Creating the Table

Created an enumerated datatype named weekday for the `day_of_the_week` since there there only seven possible values.

For the `incident_number` I decided to user the type `INTEGER` and set it as the primary key. The same datatype was also used to represent the `offense_code`.

Since the description has at most 58 character we decided to use the datatype `VARCHAR(100)` for representing it. This leave some margin while not being so big that we will waste a lot of memory.

The date was represented as the `DATE` datatype. Finally, for the latitude and longitude we used `DECIMAL` datatypes.

In [9]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [10]:
# created the enumerated datatype for representing the weekday
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# created the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

## Loading the Data into the Table

Used the `copy_expert` to load the data as it is very fast and very suuccinct to use.

In [11]:

# loaded the data from boston.csv into the table boston_crimes that is in the crimes schema
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)
cur.execute("SELECT * FROM crimes.boston_crimes")
# printed the number of rows to ensure that they were loaded
print(len(cur.fetchall()))

298329


## Revoked Public Privileges

I revoked all privileges of the `public` group on the `public` schema to ensure that users will not inherit privileges on that schema such as the ability to create tables in the `public` schema.

I also needed to revoke all privileges in the newly created schema. Doing this also makes it so that we do not need to revoke the privileges when we create users and groups because unless specified otherwise, privileges are not granted by default.

In [12]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

## Creating the read only group¶

I created a `readonly` group with `NOLOGIN` because it is a group and not a user. I granted the group the ability to connect to the `crime_db` and the ability to use the `crimes` schema.

Then I deal wit tables privileges by granting `SELECT`. I also added an extra line compared with what was asked. This extra line changes the way that privileges are given by default to the `readonly` group on new table that are created on the `crimes` schema. As I mentioned, by default not privileges are given. However I changed so that by default any user in the `readonly` group can issue select commands.

In [13]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")


## Creating the read-write group¶

I created a `readwrite` group with `NOLOGIN` because it is a group and not a user. I granted the group the ability to connect to the `crime_db` and the ability to use the `crimes` schema.

Then I deal with tables privileges by granting `SELECT, INSERT, UPDATE and DELETE`. As before I changed the default privileges so that user in the `readwrite` group have these privileges if I ever create a new table on the `crimes` schema.

In [14]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite;")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

## Creating one user for each group¶

I created a user named `data_analyst` with password `secret1` in the `readonly` group.

I created a user named `data_scientist` with password `secret2` in the `readwrite` group.

In [15]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst;")

cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist;")

## Test the database setup

Test the database setup using SQL queries on the `pg_roles` table and `information_schema.table_privileges`.

In the `pg_roles` table we will check database related privileges and for that we will look at the following columns:

* `rolname`: The name of the user / group that the privilege refers to.
* `rolsuper`: Whether this user / group is a super user. It should be set to `False` on every user / group that we have created.
* `rolcreaterole`: Whether user / group can create users, groups or roles. It should be `False` on every user / group that we have created.
* `rolcreatedb`: Whether user / group can create databases. It should be `False` on every user / group that we have created.
* `rolcanlogin`: Whether user / group can login. It should be `True` on the users and `False` on the groups that we have created.

In the `information_schema.table_privileges` we will check privileges related to SQL queries on tables. We will list the privileges of each group that we have created.

In [16]:
# close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

('readonly', False, False, False, False)
('readwrite', False, False, False, False)
('data_analyst', False, False, False, True)
('data_scientist', False, False, False, True)

('readonly', 'SELECT')
('readwrite', 'INSERT')
('readwrite', 'SELECT')
('readwrite', 'UPDATE')
('readwrite', 'DELETE')
